# Snapp Support RAG

Here we will try to implement a RAG over the support documents from Snapp's support team

In [2]:
pip install --quiet ollama sentence-transformers protobuf tqdm

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --quiet langchain langchain-community chromadb

Note: you may need to restart the kernel to use updated packages.


## Data Loading

In [4]:
from langchain.document_loaders import BSHTMLLoader,DirectoryLoader
import os

path = './supportdocs'
loader = DirectoryLoader(path,glob="**/*.html",loader_cls=BSHTMLLoader)
docs = loader.load()
print(len(docs))

203


In [9]:
print(list(map(lambda x: x.metadata['source'], docs[0:10])))

['supportdocs/معرفی-طرح-تشویقی-با-پاداش-نقدی-بر-اساس-درآمد.html', 'supportdocs/امکان-تغییر-مشخصات-خودرو-در-اپلیکیشن-کاربران-راننده.html', 'supportdocs/مواردی-که-کاربر-راننده-اسنپ-باید-در-انجام-سفر-رعایت-کند.html', 'supportdocs/فعال-شدن-قابلیت-تماس-امن-در-سفرهای-اسنپ\u200cباکس.html', 'supportdocs/راهنمای-گام\u200cبه\u200cگام-مراحل-انجام-سفر-بین\u200cاستانی-ویژه-کاربران-راننده-اسنپ.html', 'supportdocs/استفاده-از-قابلیت-گفت\u200cوگوی-متنی-درون-برنامه-چه-مزایایی-دارد؟.html', 'supportdocs/هرآنچه-باید-درباره-فرسودگی-تایر-موتور-سیکلت-بدانیم.html', 'supportdocs/هر-آنچه-باید-درباره-سهمیه-سوخت-بدانید.html', 'supportdocs/نحوه-ارسال-مدارک-در-بخش-تکمیل-مدارک-اپلیکیشن-راننده-به-همراه-ویدیوی-آموزشی.html', 'supportdocs/دلایل-کم-شدن-گرمای-بخاری-خودرو.html']


In [5]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=384)
splits = token_splitter.split_documents(docs)
print(len(splits))

/home/qasim/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1283


In [6]:
%%time
from langchain_community.embeddings import OllamaEmbeddings
embeddingModel = 'mxbai-embed-large'

embeddings = OllamaEmbeddings(model=embeddingModel,show_progress=True)

CPU times: user 150 ms, sys: 13.9 ms, total: 164 ms
Wall time: 214 ms


In [7]:
%%time
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(collection_name='snappsupport',documents=splits, embedding=embeddings,persist_directory='./chromadb')

OllamaEmbeddings: 100%|█████████████████████| 1283/1283 [01:49<00:00, 11.76it/s]


CPU times: user 4.12 s, sys: 363 ms, total: 4.48 s
Wall time: 1min 50s


## Retrieval

First we query chromadb directly to check if things are all right.

In [9]:
query = "پاداش-نقدی-بر-اساس-درآمد"
#print(token_splitter.count_tokens(query))
qembed = embeddings.embed_query(query)
results = vectorstore._collection.query(query_embeddings = qembed, n_results=5)
retrieved_documents = results['documents'][0]
print(results)

OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 14.20it/s]

{'ids': [['fe5ec7d6-1da0-11ef-8b38-8c53e6c197e3', '84167e1e-1db5-11ef-b61f-8c53e6c197e3', '84169548-1db5-11ef-b61f-8c53e6c197e3', 'fe5edf3c-1da0-11ef-8b38-8c53e6c197e3', 'fe5ed29e-1da0-11ef-8b38-8c53e6c197e3']], 'distances': [[296.3659362792969, 296.3659362792969, 306.24133978363614, 306.2413635253906, 307.16693115234375]], 'metadatas': [[{'source': 'supportdocs/حفظ-خونسردی-در-ترافیک.html', 'title': 'حفظ خونسردی در ترافیک!'}, {'source': 'supportdocs/حفظ-خونسردی-در-ترافیک.html', 'title': 'حفظ خونسردی در ترافیک!'}, {'source': 'supportdocs/ویژگی-جدید-اپلیکیشن-راننده-اسنپ-نمایش-دلایل-مسدودی-حساب-کاربری.html', 'title': 'ویژگی جدید اپلیکیشن راننده اسنپ: نمایش دلایل مسدودی حساب کاربری'}, {'source': 'supportdocs/ویژگی-جدید-اپلیکیشن-راننده-اسنپ-نمایش-دلایل-مسدودی-حساب-کاربری.html', 'title': 'ویژگی جدید اپلیکیشن راننده اسنپ: نمایش دلایل مسدودی حساب کاربری'}, {'source': 'supportdocs/راهنمای-جامع-نگهداری-از-خودرو-مباحث-مرتبط-با-تعویض-روغن،-فیلتر-و-مایعات-خودرو.html', 'title': 'راهنمای جامع نگهداری

In [10]:
#ollamaBase = 'http://ollama-alibo-gpu-testing.apps.private.okd4.teh-2.snappcloud.io/'
ollamaBase = 'http://localhost:11434'
import ollama

def newOllamaClient(baseURL):
    return ollama.Client(baseURL)

client = newOllamaClient(baseURL=ollamaBase)
retriever = vectorstore.as_retriever()

In [11]:
llmModel = 'llama3'

systemPrompt = """You are a helpful expert assistant conversant in persian. Your users are drivers asking questions about Snapp, a ride hailing company.
                You will be shown the user's question, and  relevant information from the documentation as context.
                Answer the user's question in Persian using only the context and cite your sources"""


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = client.chat(model=llmModel, options = { 'temperature': 0}, messages=[{'role': 'system', 'content': systemPrompt},{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    print("using from context",list(map(lambda x: x.metadata['source'],retrieved_docs)))
    return ollama_llm(question, formatted_context)

In [12]:
%%time
query = 'آخرین سفر من انجام نشده ولی تو لیست سفرها پایان خورده، چرا از من کمیسون کسر شده؟'
result = rag_chain(query)
print(result)

OllamaEmbeddings: 100%|███████████████████████████| 1/1 [00:00<00:00, 16.53it/s]


using from context ['supportdocs/مزایای-فعالیت-به\u200c-صورت-پاره\u200cوقت-در-اسنپ.html', 'supportdocs/مزایای-فعالیت-به\u200c-صورت-پاره\u200cوقت-در-اسنپ.html', 'supportdocs/داستان\u200cهای-آقای-اسنپیان؛-خرید-گوشی-موبایل-برای-فرزند.html', 'supportdocs/داستان\u200cهای-آقای-اسنپیان؛-خرید-گوشی-موبایل-برای-فرزند.html']
جواب: آخرین سفر من انجام نشده ولی تو لیست سفرها پایان خورده، چرا از من کمیسون کسر شده؟

به نظر می‌رسد که شما آخرین سفر خود را ثبت نکرده‌اید اما در لیست سفرهای خود پایان یافته است. این به معنای آن است که سهمیه بنزین شما به دلیل پیمایش ماهانه بیش از [UNK] کیلومتر، مشمول دریافت اعتبار «سهمیه بنزنین» می‌شود. بنابراین، باید در سامانه سماس ثبت نام کنید تا بتوانید از این سهمیه بهره‌مند شوید.

منبع: مقاله «نحوه مشاهده طرحهای تشویقی در اپلیکیشن رانندگان» و «نحوه ثبت اطلاعات در سامانه ملی اطلاعات سفر، سماس»
CPU times: user 42.1 ms, sys: 6.23 ms, total: 48.3 ms
Wall time: 9.78 s
